In [ ]:
import pynmea2
import pandas as pd

DATA_FILE = "../data/walk_10_19.txt"


# https://receiverhelp.trimble.com/alloy-gnss/en-us/NMEA-0183messages_GST.html
# https://receiverhelp.trimble.com/alloy-gnss/en-us/NMEA-0183messages_RMC.html
#$GPGST,182330.00,18,,,,5.7,3.3,12*7A
#$GPRMC,182331.00,A,3750.81870,N,12216.72288,W,0.092,,191025,,,A*6B

std_data = {'timestamp':[], 'std_dev_latitude':[]}
pose_data = {'timestamp':[], 'lat':[], 'lon':[], 'vel':[]}

with open(DATA_FILE, 'r') as fd:
    for line in fd.readlines():
        try:
            msg = pynmea2.parse(line)
            #print(repr(msg))
            if isinstance(msg, pynmea2.types.talker.RMC):
                pose_data['timestamp'].append(msg.timestamp)
                pose_data['lat'].append(msg.latitude)
                pose_data['lon'].append(msg.longitude)
                pose_data['vel'].append(msg.spd_over_grnd)
            if isinstance(msg, pynmea2.types.talker.GST):
                std_data['timestamp'].append(msg.timestamp)
                std_data['std_dev_latitude'].append(msg.std_dev_latitude)
        except pynmea2.ParseError as e:
            print('Parse error: {}'.format(e))
            continue

pose_df = pd.DataFrame(pose_data)


In [ ]:
# Convert collected pose_data into a DataFrame and plot relative trajectory
import plotly.express as px
import math


# Reference (start) point
lat0 = pose_df.loc[0, 'lat']
lon0 = pose_df.loc[0, 'lon']

# Approximate conversion: meters per degree latitude ~= 111132, longitude scaled by cos(lat)
meters_per_deg_lat = 111132.0
meters_per_deg_lon = 111320.0 * math.cos(math.radians(lat0))

pose_df['x_m'] = (pose_df['lon'] - lon0) * meters_per_deg_lon
pose_df['y_m'] = (pose_df['lat'] - lat0) * meters_per_deg_lat

# Create a simple scatter plot of trajectory (x east, y north)
fig = px.scatter(pose_df, x='x_m', y='y_m', color='vel',
                    title='Trajectory relative to start (meters)',
                    labels={'x_m': 'East (m)', 'y_m': 'North (m)', 'vel': 'Speed (knots)'} ,
                    hover_data=['lat', 'lon', 'vel'])
fig.update_traces(mode='lines+markers')
fig.update_layout(yaxis=dict(scaleanchor='x', scaleratio=1))
fig.show()

In [ ]:
# Convert collected pose_data into a DataFrame and plot relative trajectory
import plotly.express as px


pose_df['vel_mean'] = pose_df['vel'].rolling(window=5, min_periods=1, center=False).mean()




# Create a simple scatter plot of trajectory (x east, y north)
fig = px.scatter(pose_df, x='timestamp', y='vel_mean')
fig.update_traces(mode='lines+markers')
fig.show()